In [19]:
import random

# **Input Nodes**

In [20]:
class InputNode:

    def __init__(self):
        self.activation = 0.0
        self.forward = []

In [21]:
def make_input_nodes(num_of_features):
    return [InputNode() for _ in range(num_of_features)]

In [22]:
input_nodes = make_input_nodes(3)
print(len(input_nodes))

3


# **Connections**

In [23]:
class Connection:

    def __init__(self):
        self.activation = 0.0
        self.backward = None
        self.forward = None
        self.weight = 0.0

In [24]:
def make_connections(previous_layer, width):
    connections = []
    for node in previous_layer:
        node.forward = [Connection() for _ in range(width)]
        for connection in node.forward:
            connection.backward = node
        connections.extend(node.forward)
    for connection in connections:
        connection.weight = random.uniform(-0.2, 0.2)
    return connections

In [25]:
connections = make_connections(input_nodes, 3)
print(len(connections))
print(connections[0].weight)
print(type(connections[0].backward))

9
0.11242010044297723
<class '__main__.InputNode'>


# **Neurons**

In [26]:
class Neuron:

    def __init__(self):
        self.activation = 0.0
        self.backward = []
        self.forward = None
        self.bias = 0.0

In [27]:
def make_neurons(previous_layer, width):
    neurons = [Neuron() for _ in range(width)]
    nodes = list({connection.backward for connection in previous_layer})
    for node in nodes:
        for connection, neuron in zip(node.forward, neurons):
            connection.forward = neuron
            neuron.backward.append(connection)
    for neuron in neurons:
        neuron.bias = 0.1
    return neurons

In [28]:
neurons = make_neurons(connections, 3)
print(len(neurons))
print(neurons[0].bias)
print(neurons[0].backward)
print(neurons[0].backward[0].forward == neurons[0])

3
0.1
[<__main__.Connection object at 0x7a0d49b60a90>, <__main__.Connection object at 0x7a0d4049dad0>, <__main__.Connection object at 0x7a0d4047e290>]
True


# **Nonlinear Nodes**

In [29]:
class NonlinearNode:

    def __init__(self):
        self.activation = 0.0
        self.backward = None
        self.forward = []

In [30]:
def make_nonlinear_nodes(previous_layer, width):
    nonlinear_nodes = [NonlinearNode() for _ in range(width)]
    for neuron, nonlinear_node in zip(previous_layer, nonlinear_nodes):
        neuron.forward = nonlinear_node
        nonlinear_node.backward = neuron
    return nonlinear_nodes

In [31]:
nonlinear_nodes = make_nonlinear_nodes(neurons, 3)
print(len(nonlinear_nodes))
print(nonlinear_nodes[0].backward)
print(nonlinear_nodes[0].backward.forward == nonlinear_nodes[0])

3
True


# **Output Nodes**

In [32]:
class OutputNode:

    def __init__(self):
        self.activation = 0.0
        self.backward = None

In [33]:
def make_output_nodes(previous_layer, num_of_outputs):
    output_nodes = [OutputNode() for _ in range(num_of_outputs)]
    for nonlinear_node, output_node in zip(previous_layer, output_nodes):
        nonlinear_node.forward = output_node
        output_node.backward = nonlinear_node
    return output_nodes

In [34]:
output_nodes = make_output_nodes(nonlinear_nodes, 3)
print(len(output_nodes))
print(output_nodes[0].backward)
print(output_nodes[0].backward.forward == output_nodes[0])

3
True


# **Bringing it Together**

In [35]:
def construct_model(shape):
    layers = [make_input_nodes(shape[0])]
    for width in shape[1:]:
        layers.append(make_connections(layers[-1], width))
        layers.append(make_neurons(layers[-1], width))
        layers.append(make_nonlinear_nodes(layers[-1], width))
    layers.append(make_output_nodes(layers[-1], shape[-1]))
    return layers

# **Testing**

In [36]:
shape = [3, 3, 3]
model = construct_model(shape)
print(f'Nodes in network: {[len(layer) for layer in model]}')

Nodes in network: [3, 9, 3, 3, 9, 3, 3, 3]
